In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

batch_size = 256
learning_rate = 0.0002
num_epoch = 10

In [4]:
mnist_train = dset.MNIST("./", train = True, transform = transforms.ToTensor(), target_transform=None, download=True)
mnist_test = dset.MNIST(",/", train = False, transform = transforms.ToTensor(), target_transform=None, download = True)

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size = batch_size, shuffle = True, num_workers=2, drop_last = True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size = batch_size, shuffle=False, num_workers=2, drop_last=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 79666033.52it/s]


Extracting ,/MNIST/raw/train-images-idx3-ubyte.gz to ,/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 24736715.10it/s]

Extracting ,/MNIST/raw/train-labels-idx1-ubyte.gz to ,/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 24168002.39it/s]


Extracting ,/MNIST/raw/t10k-images-idx3-ubyte.gz to ,/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 2868191.62it/s]


Extracting ,/MNIST/raw/t10k-labels-idx1-ubyte.gz to ,/MNIST/raw



In [5]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer = nn.Sequential(
        nn.Conv2d(1,16,5),
        nn.ReLU(),
        nn.Conv2d(16,32,5),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
        nn.Conv2d(32,64,5),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )

    self.fc_layer = nn.Sequential(
        nn.Linear(64*3*3, 100),
        nn.ReLU(),
        nn.Linear(100, 10)
    )

  def forward(self, x):
    out = self.layer(x)
    out = out.view(batch_size, -1)
    out = self.fc_layer(out)
    return out

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [8]:
loss_arr = []
for i in range(num_epoch):
  for j, [image, label] in enumerate(train_loader):
    x = image.to(device)
    y_ = label.to(device)

    optimizer.zero_grad()
    output = model.forward(x)
    loss = loss_func(output, y_)
    loss.backward()
    optimizer.step()

    if j%1000 == 0:
      print(loss)
      loss_arr.append(loss.cpu().detach().numpy())

tensor(2.2994, grad_fn=<NllLossBackward0>)
tensor(0.2591, grad_fn=<NllLossBackward0>)
tensor(0.1830, grad_fn=<NllLossBackward0>)
tensor(0.1590, grad_fn=<NllLossBackward0>)
tensor(0.2005, grad_fn=<NllLossBackward0>)
tensor(0.0663, grad_fn=<NllLossBackward0>)
tensor(0.0642, grad_fn=<NllLossBackward0>)
tensor(0.0572, grad_fn=<NllLossBackward0>)
tensor(0.0745, grad_fn=<NllLossBackward0>)
tensor(0.0216, grad_fn=<NllLossBackward0>)


In [9]:
correct = 0
total = 0

with torch.no_grad():
  for image, label in test_loader:
    x = image.to(device)
    y_ = label.to(device)

    otput = model.forward(x)
    _, output_index = torch.max(output, 1)

    total += label.size(0)
    correct +=(output_index ==y_).sum().float()
  print("Accuracy of Test Data: {}".format(100*correct/total))

Accuracy of Test Data: 10.036057472229004
